# Ambulance Dispatching Mathematical Modeling Formulation

<div class="alert-danger">
# HAVEN'T CHECKED IT IN DETAIL YET
</div>

In [8]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB, Model, quicksum

from collections import defaultdict
from itertools import product # this is used to generate all possible states

In [ ]:
# gurobi_mdp_lp.py
def build_mdp_lp(
    S,                      # iterable of states
    CallType,               # iterable of (h, i) tuples
    X,                      # dict: X[(s,(h,i))] -> iterable of actions j
    u,                      # dict: u[(h,i,j)] -> reward u_ij^h (float)
    P                       # dict: P[((s,j,(h,i)), (s_prime,(h_prime,i_prime)))] -> prob
):
    m = Model("MDP_OccupancyLP")
    m.Params.OutputFlag = 0  # quiet by default; set to 1 for logs

    # Decision variables y[s,j,(h,i)] >= 0 only for feasible (s,(h,i),j)
    y = {}
    for s in S:
        for ct in CallType:
            key = (s, ct)
            for j in X.get(key, []):
                varname = f"y[{s},{j},{ct}]"
                y[(s, j, ct)] = m.addVar(lb=0.0, name=varname)
    m.update()

    # Objective: sum u_ij^h * y
    m.setObjective(
        quicksum(u.get((ct[0], ct[1], j), 0.0) * y[(s, j, ct)]
                 for (s, j, ct) in y.keys()),
        GRB.MAXIMIZE
    )

    inbound = defaultdict(list)  # (s_prime, ct_prime) -> list of ((s,j,ct), prob)
    for ((s, j, ct), (sp, ctp)) , prob in P.items():
        if prob != 0.0:
            inbound[(sp, ctp)].append(((s, j, ct), prob))

    for sp in S:
        for ctp in CallType:
            lhs = quicksum(y[(sp, jp, ctp)] for jp in X.get((sp, ctp), []))
            rhs = quicksum(prob * y[(s, j, ct)]
                           for ((s, j, ct), prob) in inbound.get((sp, ctp), []))
            m.addConstr(lhs - rhs == 0.0, name=f"flow[{sp},{ctp}]")

    # Normalization: sum y = 1
    m.addConstr(quicksum(y[v] for v in y) == 1.0, name="normalization")

    m.update()
    return m, y

### States

In [12]:
def stateGenerator(n, m): # n: number of locations, m: number of ambulances
    states  = product(range(n + 1), repeat=m)
    return list(states)

### Call Types

In [13]:
def callTypeGenerator(n, priorityLevels):
    calltypes = product(range(n), range(priorityLevels))
    return list(calltypes)

### Actions

In [14]:
def actionGenerator(n, m):
    states = stateGenerator(n, m)
    calls = callTypeGenerator(n, 2)
    # dict: X[(s,(h,i))] -> iterable of actions j
    X = {}
    for s in states:
        for (h,i) in calls:
            key = (s, (h,i))
            X[key] = []
            for j in range(m):
                if s[j] == 0:
                    X[key].append(j)
    return X

in order to test the generator functions 

In [21]:
if True:  # Change to True to test the generator functions
    XX = actionGenerator(6, 2)  # Example for 6 locations and 2 ambulances
    for s in stateGenerator(6, 2):  # Example for 6 locations and 2 ambulances
        for ct in callTypeGenerator(6, 2):  # Example for 6 locations and 2 priority levels
            key = (s, ct)
            for j in XX.get(key, []):
                varname = f"y[{s},{j},{ct}]"
                print(varname)

y[(0, 0),0,(0, 0)]
y[(0, 0),1,(0, 0)]
y[(0, 0),0,(0, 1)]
y[(0, 0),1,(0, 1)]
y[(0, 0),0,(1, 0)]
y[(0, 0),1,(1, 0)]
y[(0, 0),0,(1, 1)]
y[(0, 0),1,(1, 1)]
y[(0, 0),0,(2, 0)]
y[(0, 0),1,(2, 0)]
y[(0, 0),0,(2, 1)]
y[(0, 0),1,(2, 1)]
y[(0, 0),0,(3, 0)]
y[(0, 0),1,(3, 0)]
y[(0, 0),0,(3, 1)]
y[(0, 0),1,(3, 1)]
y[(0, 0),0,(4, 0)]
y[(0, 0),1,(4, 0)]
y[(0, 0),0,(4, 1)]
y[(0, 0),1,(4, 1)]
y[(0, 0),0,(5, 0)]
y[(0, 0),1,(5, 0)]
y[(0, 0),0,(5, 1)]
y[(0, 0),1,(5, 1)]
y[(0, 1),0,(0, 0)]
y[(0, 1),0,(0, 1)]
y[(0, 1),0,(1, 0)]
y[(0, 1),0,(1, 1)]
y[(0, 1),0,(2, 0)]
y[(0, 1),0,(2, 1)]
y[(0, 1),0,(3, 0)]
y[(0, 1),0,(3, 1)]
y[(0, 1),0,(4, 0)]
y[(0, 1),0,(4, 1)]
y[(0, 1),0,(5, 0)]
y[(0, 1),0,(5, 1)]
y[(0, 2),0,(0, 0)]
y[(0, 2),0,(0, 1)]
y[(0, 2),0,(1, 0)]
y[(0, 2),0,(1, 1)]
y[(0, 2),0,(2, 0)]
y[(0, 2),0,(2, 1)]
y[(0, 2),0,(3, 0)]
y[(0, 2),0,(3, 1)]
y[(0, 2),0,(4, 0)]
y[(0, 2),0,(4, 1)]
y[(0, 2),0,(5, 0)]
y[(0, 2),0,(5, 1)]
y[(0, 3),0,(0, 0)]
y[(0, 3),0,(0, 1)]
y[(0, 3),0,(1, 0)]
y[(0, 3),0,(1, 1)]
y[(0, 3),0,(

In [ ]:

u = {}
for s in stateGenerator(6, 2): 
    for ct in callTypeGenerator(6, 2):  # Example for 6 locations and 2 priority levels
        key = (s, ct)
        for j in actionGenerator(6,2).get(key, []):
            i = ct[0]  # h is the first element of the call type tuple
            h = ct[1]  # i is the second element of the call type tuple
            u[(i,h, j)] = 1.0
            print("i: ",i, "h: ",h, "j: ", j, "u: ", u[(i,h,j)])


i:  0 h:  0 j:  0 u:  1.0
i:  0 h:  0 j:  1 u:  1.0
i:  0 h:  1 j:  0 u:  1.0
i:  0 h:  1 j:  1 u:  1.0
i:  1 h:  0 j:  0 u:  1.0
i:  1 h:  0 j:  1 u:  1.0
i:  1 h:  1 j:  0 u:  1.0
i:  1 h:  1 j:  1 u:  1.0
i:  2 h:  0 j:  0 u:  1.0
i:  2 h:  0 j:  1 u:  1.0
i:  2 h:  1 j:  0 u:  1.0
i:  2 h:  1 j:  1 u:  1.0
i:  3 h:  0 j:  0 u:  1.0
i:  3 h:  0 j:  1 u:  1.0
i:  3 h:  1 j:  0 u:  1.0
i:  3 h:  1 j:  1 u:  1.0
i:  4 h:  0 j:  0 u:  1.0
i:  4 h:  0 j:  1 u:  1.0
i:  4 h:  1 j:  0 u:  1.0
i:  4 h:  1 j:  1 u:  1.0
i:  5 h:  0 j:  0 u:  1.0
i:  5 h:  0 j:  1 u:  1.0
i:  5 h:  1 j:  0 u:  1.0
i:  5 h:  1 j:  1 u:  1.0
i:  0 h:  0 j:  0 u:  1.0
i:  0 h:  1 j:  0 u:  1.0
i:  1 h:  0 j:  0 u:  1.0
i:  1 h:  1 j:  0 u:  1.0
i:  2 h:  0 j:  0 u:  1.0
i:  2 h:  1 j:  0 u:  1.0
i:  3 h:  0 j:  0 u:  1.0
i:  3 h:  1 j:  0 u:  1.0
i:  4 h:  0 j:  0 u:  1.0
i:  4 h:  1 j:  0 u:  1.0
i:  5 h:  0 j:  0 u:  1.0
i:  5 h:  1 j:  0 u:  1.0
i:  0 h:  0 j:  0 u:  1.0
i:  0 h:  1 j:  0 u:  1.0
i:  1 h:  0 

### Rewards

In [ ]:
# u = { (h,i,j): u_hij, ... }
def rewardGenerator(n,m):
    u = {}
    for s in stateGenerator(n, m): 
        for ct in callTypeGenerator(n, m):  # Example for 6 locations and 2 priority levels
            key = (s, ct)
            for j in actionGenerator(n,m).get(key, []):
                i = ct[0]  # h is the first element of the call type tuple
                h = ct[1]  # i is the second element of the call type tuple
                u[(i,h, j)] = 1.0
                #print("i: ",i, "h: ",h, "j: ", j, "u: ", u[(i,h,j)])
    return u

In [ ]:
# 1) Define sets
S = [...]                       # list of server-configuration states
CallType = [...]                # list of (h,i) pairs
X = { (s,(h,i)) : [...] }       # actions available (server indices)

# 2) Rewards
u = { (h,i,j): u_hij, ... }

# 3) Transition probs (uniformized)
# Map ((s,j,(h,i)), (s_prime,(h_prime,i_prime))) -> p
P = { ((s,j,(h,i)), (s2,(h2,i2))) : prob, ... }

# 4) Build/solve
from gurobi_mdp_lp import build_mdp_lp, extract_policy
model, y = build_mdp_lp(S, CallType, X, u, P)
model.optimize()

# 5) Policy extraction (stationary randomized)
pi = extract_policy(y, X)

In [9]:
def extract_policy(y_vars, X):
    """
    Given optimized y and the action sets X, compute a stationary randomized policy:
      q(s,(h,i))[j] = y(s,j,(h,i)) / sum_{j' in X(s,(h,i))} y(s,j',(h,i))
    Returns dict: policy[(s,(h,i))] -> dict {j: prob}
    """
    # Sum per (s,ct)
    denom = defaultdict(float)
    for (s, j, ct), var in y_vars.items():
        denom[(s, ct)] += var.X

    policy = {}
    for (s, ct), total in denom.items():
        choices = X.get((s, ct), [])
        pi = {}
        if total > 0:
            for j in choices:
                y_key = (s, j, ct)
                if y_key in y_vars:
                    pi[j] = y_vars[y_key].X / total
        else:
            # unreachable (s,ct): put zero probs
            for j in choices:
                pi[j] = 0.0
        policy[(s, ct)] = pi
    return policy